## Instructions
- Attach notebook to a cluster (Serverless preferred).
- Run Cells 2, 3 and 4 for the input widget parameters to get polpulated.
- Fill in the parameters. Select the dashboard to extract and provide the catalog and schema that were used while deploying the dashboards.
- Run the rest of the notebook to extract the dashboard json into the current directory.

In [0]:
%pip install databricks-sdk==0.38.0
dbutils.library.restartPython()

In [0]:
import json
from pathlib import Path
import os
import re

from databricks.sdk import WorkspaceClient
from databricks.sdk.service import jobs

# Client initialized for the current workspace
w = WorkspaceClient()

# List all the JSON files from the current folder
def list_json_dash_files():
    notebook_folder = json.loads(
        dbutils.notebook.entry_point.getDbutils().notebook().getContext().safeToJson()
    )["attributes"]["notebook_path"]
    new_folder_name = "dashboard_assess_dbx_costs"
    dashboard_save_path = (
        f'{notebook_folder.rsplit("/", 1)[0]}/dashboard_assess_dbx_costs'
    )
    Path(new_folder_name).mkdir(parents=True, exist_ok=True)
    json_files = [
        f for f in os.listdir(".") if os.path.isfile(f) and f.endswith(".lvdash.json")
    ]
    print(f"Dashboard JSON files found: {json_files}")
    return json_files, dashboard_save_path


def extract_dashboard(selected_dashboard, catalog, schema):
    dashboard_id = w.workspace.get_status(
        f"{dashboard_save_path}/{selected_dashboard}"
    ).resource_id
    dash_details = w.lakeview.get(dashboard_id).serialized_dashboard
    replaced_dash = dash_details.replace(f"{catalog}", "{catalog}").replace(
        f"{schema}", "{schema}"
    )
    urls = re.findall("(https?://\S+/dashboard\S+/published)", replaced_dash)
    for url in urls:
        replaced_dash = replaced_dash.replace(f"{url}", "*")
    parsed_file = json.loads(replaced_dash)

    with open(f"{selected_dashboard}", "w") as json_file:
        json_file.write(json.dumps(parsed_file, indent=4))
    print(f'Dashboard "{selected_dashboard}" extracted successfully')


json_files, dashboard_save_path = list_json_dash_files()

In [0]:
dbutils.widgets.dropdown('selected_dashboard', json_files[0], json_files, 'Dashboard to extract') # Select the dashboard to extract
dbutils.widgets.text('catalog', 'main') # Provide the catalog that was used when the dashboard was created
dbutils.widgets.text('schema', 'default') # Provide the schema that was used when the dashboard was created

In [0]:
selected_dashboard = dbutils.widgets.get('selected_dashboard')
catalog = dbutils.widgets.get('catalog')
schema = dbutils.widgets.get('schema')

In [0]:
print("Extracting dashboard...")
extract_dashboard(selected_dashboard, catalog, schema)